In [1]:
R1=[0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.2,1.4,1.6,1.8,2.0,2.2,2.4,2.6,2.8,3.0,4.0,5.0]
R2=[0.6,0.61,0.62,0.63,0.64,0.65,0.66,0.67,0.68,0.69,0.7,0.71,0.72,0.73,0.74,0.75,0.76,0.77,0.78,0.79,0.8]

In [2]:
# import common packages
import numpy as np
from qiskit import Aer

# lib from Qiskit Aqua
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.components.optimizers import COBYLA


# lib from Qiskit Aqua Chemistry
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PSI4Driver
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry import MP2Info

In [3]:
    psi4_h2_config = '''
molecule mol {{
   0 1
   H   0.0 0.0 0.0
   H   {} 0.0 0.0
   symmetry c1
}}

set {{
      basis sto-3g
      scf_type pk
      reference uhf
      guess huckel
      guess_mix true
      
}}
'''

In [4]:
map_type = 'jordan_wigner'
qubit_reduction = False

In [5]:
HF_1=[]
nuclear_repulsion_energy1=[]
Ham1=[]
for i in range(len(R1)):
    length=R1[i]
    driver = PSI4Driver(config=psi4_h2_config.format(length)) 
    molecule=driver.run()
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    nuclear_repulsion_energy1.append(molecule.nuclear_repulsion_energy)
    HF_1.append(molecule.hf_energy)
    ferOp = FermionicOperator(h1=h1, h2=h2)
    qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
    qubitOp.chop(10**-10)
    Ham1.append(qubitOp)

In [6]:
num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))

# of electrons: 2
# of spin orbitals: 4


In [7]:
# setup COBYLA optimizer
max_eval = 2000
optimizer = COBYLA(maxiter=max_eval)

# setup HartreeFock state
HF_state = HartreeFock(num_orbitals=num_spin_orbitals, num_particles=num_particles,qubit_mapping=map_type,two_qubit_reduction=False)


# setup UCCSD variational form
uccsd = UCCSD(num_orbitals=num_spin_orbitals, num_particles=num_particles, 
                 initial_state=HF_state, qubit_mapping=map_type,
                 two_qubit_reduction=qubit_reduction, num_time_slices=1)

# backend setting
backend = Aer.get_backend('statevector_simulator')

In [8]:
sto_uccsd=[]
sto_diagext=[]
for i in range(len(Ham1)):
    input_operator=Ham1[i]
    sto_diagext.append(float(NumPyEigensolver(input_operator).run().eigenvalues.real))
    vqe = VQE(operator=input_operator, optimizer=optimizer, quantum_instance=backend)
    vqe.var_form = uccsd
    result = vqe.compute_minimum_eigenvalue()
    sto_uccsd.append(result.eigenvalue.real)
    print('VQE:', result.eigenvalue.real)    

VQE: -2.3657277306921145
VQE: -2.237092704769789
VQE: -2.1135142085585557
VQE: -1.9982479967770612
VQE: -1.8921568759770844
VQE: -1.7956191703728566
VQE: -1.7085349545200246
VQE: -1.6303275230453362
VQE: -1.4977217438227195
VQE: -1.3934519672095331
VQE: -1.3142084799515437
VQE: -1.255804274930285
VQE: -1.2132296916647567
VQE: -1.1817588755497859
VQE: -1.1577451600248232
VQE: -1.1387241160032324
VQE: -1.123139878003043
VQE: -1.1099901577772457
VQE: -1.0654618776021096
VQE: -1.0389991574514639


In [9]:
import pandas as pd
HF=np.array(HF_1)-np.array(nuclear_repulsion_energy1)

dic = {
    "H-H Length": R1, 
    "HF": HF,
    "Ext_DIAG": sto_diagext,
    "UCCSD_VQE": sto_uccsd,
}
df = pd.DataFrame(dic)
df

,H-H Length,HF,Ext_DIAG,UCCSD_VQE
0,0.3,-2.357752,-2.365728,-2.365728
1,0.4,-2.227304,-2.237093,-2.237093
2,0.5,-2.101351,-2.113514,-2.113514
3,0.6,-1.983090,-1.998248,-1.998248
4,0.7,-1.873316,-1.892157,-1.892157
5,0.8,-1.772322,-1.795619,-1.795619
6,0.9,-1.679889,-1.708535,-1.708535
7,1.0,-1.595286,-1.630328,-1.630328
8,1.2,-1.447354,-1.497722,-1.497722
9,1.4,-1.346475,-1.393452,-1.393452
